In [1]:
import pandas as pd
from datetime import datetime, timedelta

### Data Loading

In [2]:
def load_and_clean_data():
    """
    Load and clean datasets for further analysis.
    """
    # Load datasets
    import os
    import types
    import pandas as pd
    from botocore.client import Config
    import ibm_boto3

    def __iter__(self): 
        return 0

    # IBM Cloud Object Storage credentials
    cos_client = ibm_boto3.client(
        service_name='s3',
        ibm_api_key_id='NYhoYyxMoqRAPvjF9EFNTyOfSasRKs4dIdeW5_aZwlg9',
        ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
        config=Config(signature_version='oauth'),
        endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud'
    )

    bucket = '007tlhworkingproject-donotdelete-pr-hgvioy9vygrquk'

    # Object keys and their corresponding dataframe names
    object_keys = {
        'transactions': 'Two_Years_Customer_Portfolio_Transactions_Data.csv',
        'historical_prices': 'full_historical_prices.csv',
        'customer_profiles': 'Customer_Profile.csv',
        'tax_rates': 'Tax Rates.csv',
        'stock_expanded_data': 'stock_expanded_data.csv'
    }

    dataframes = {}

    # Reading each object into a pandas dataframe
    for df_name, object_key in object_keys.items():
        body = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']
        if not hasattr(body, "__iter__"):
            body.__iter__ = types.MethodType(__iter__, body)
        dataframes[df_name] = pd.read_csv(body)

    # Extract dataframes
    transactions = dataframes['transactions']
    historical_prices = dataframes['historical_prices']
    customer_profiles = dataframes['customer_profiles']
    tax_rates = dataframes['tax_rates']
    stock_expanded_data = dataframes['stock_expanded_data']

    # Ensure date columns are properly parsed
    transactions["Transaction Date"] = pd.to_datetime(transactions["Transaction Date"], errors="coerce")
    historical_prices["Date"] = pd.to_datetime(historical_prices["Date"], errors="coerce")

    return transactions, historical_prices, customer_profiles, tax_rates, stock_expanded_data

# Call the function and load the datasets
original_transactions, historical_prices, customer_profiles, tax_rates, stock_expanded_data = load_and_clean_data()


/tmp/wsuser/ipykernel_438/88197021.py:52: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  transactions["Transaction Date"] = pd.to_datetime(transactions["Transaction Date"], errors="coerce")
/tmp/wsuser/ipykernel_438/88197021.py:53: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  historical_prices["Date"] = pd.to_datetime(historical_prices["Date"], errors="coerce")


### Helper Functions

In [3]:
def get_latest_prices(historical_data):
    """
    Extract the latest prices for all tickers.
    """
    latest_prices = (
        historical_data.sort_values(by=["Ticker", "Date"])
        .groupby("Ticker")["Close"]
        .last()
        .to_dict()
    )
    return latest_prices

def calculate_holding_period(buy_date, sell_date):
    """
    Calculate the holding period between buy and sell dates.
    """
    return (sell_date - buy_date).days

def get_applicable_tax_rate(income, holding_period, tax_rate_table):
    """
    Determine the applicable tax rate based on income and holding period.
    """
    for _, row in tax_rate_table.iterrows():
        lower_limit = row["Income Range-Lower Limit"]
        upper_limit = row["Income Range-Upper Limit"]

        # Handle the last row with no upper limit
        if pd.isna(upper_limit) or income <= upper_limit:
            if income >= lower_limit:
                return (
                    row["Long-Term Capital Gains Tax Rate"]
                    if holding_period > 365
                    else row["Short-Term Capital Gains Tax Rate"]
                )
    
    # If income exceeds all ranges, use the last row
    last_row = tax_rate_table.iloc[-1]
    return (
        last_row["Long-Term Capital Gains Tax Rate"]
        if holding_period > 365
        else last_row["Short-Term Capital Gains Tax Rate"]
    )

### Portfolio Processing

In [4]:
def compute_fifo_portfolio_with_metrics(transactions, current_prices, customer_id):
    """
    Compute FIFO portfolio with unrealized gains/losses and metrics.
    """
    transactions_filtered = transactions[transactions["Customer ID"] == customer_id].copy()
    transactions_filtered = transactions_filtered.sort_values(by=["Ticker", "Transaction Date"])
    
    portfolio = []

    for ticker, group in transactions_filtered.groupby("Ticker"):
        purchase_lots = []
        remaining_quantity = 0
        
        for _, row in group.iterrows():
            if row["Transaction Type"] == "Buy":
                purchase_lots.append((row["Quantity"], row["Price"], row["Transaction Date"]))
                remaining_quantity += row["Quantity"]
            elif row["Transaction Type"] == "Sell":
                sell_quantity = row["Quantity"]
                while sell_quantity > 0 and purchase_lots:
                    lot_quantity, lot_price, lot_date = purchase_lots[0]
                    if sell_quantity >= lot_quantity:
                        sell_quantity -= lot_quantity
                        remaining_quantity -= lot_quantity
                        purchase_lots.pop(0)
                    else:
                        purchase_lots[0] = (lot_quantity - sell_quantity, lot_price, lot_date)
                        remaining_quantity -= sell_quantity
                        sell_quantity = 0

        if remaining_quantity > 0:
            total_cost = sum(lot[0] * lot[1] for lot in purchase_lots)
            weighted_avg_price = total_cost / remaining_quantity
            holding_periods = [
                (lot[0] / remaining_quantity) * (datetime.now() - lot[2]).days for lot in purchase_lots
            ]
            avg_holding_period = sum(holding_periods)
            current_price = current_prices.get(ticker, 0)
            unrealized_gain_loss = (current_price - weighted_avg_price) * remaining_quantity

            portfolio.append({
                "Ticker": ticker,
                "Quantity": remaining_quantity,
                "Purchase Price": weighted_avg_price,
                "Current Price": current_price,
                "Unrealized Gain/Loss": unrealized_gain_loss,
                "Holding Period (Days)": avg_holding_period
            })

    return pd.DataFrame(portfolio)

### Tax Loss Harvesting

In [5]:
def identify_tlh_candidates(portfolio):
    """
    Identify TLH candidates with unrealized losses.
    """
    return portfolio[portfolio["Unrealized Gain/Loss"] < 0].sort_values("Unrealized Gain/Loss")

def calculate_tax_savings(tlh_candidates, income, tax_rate_table):
    """
    Calculate tax savings for TLH candidates.
    """
    tlh_candidates["Tax Rate"] = tlh_candidates.apply(
        lambda row: get_applicable_tax_rate(income, row["Holding Period (Days)"], tax_rate_table),
        axis=1
    )
    tlh_candidates["Tax Savings"] = -tlh_candidates["Unrealized Gain/Loss"] * tlh_candidates["Tax Rate"]
    return tlh_candidates

def finalize_tlh_recommendation(tlh_candidates, realized_gains, max_loss_offset=3000):
    total_offset_limit = realized_gains + max_loss_offset if realized_gains > 0 else max_loss_offset
    remaining_offset = total_offset_limit

    tlh_candidates["Tax Savings per Loss"] = tlh_candidates["Tax Savings"] / abs(tlh_candidates["Unrealized Gain/Loss"])
    tlh_candidates = tlh_candidates.sort_values(
        by=["Tax Savings", "Tax Savings per Loss"], ascending=[False, False]
    )
    tlh_candidates["Recommended"] = False
    tlh_candidates["Partial Quantity"] = 0

    for i, row in tlh_candidates.iterrows():
        if remaining_offset <= 0:
            break

        # Calculate unrealized loss for the recommended quantity
        unrealized_loss_per_unit = abs(row["Unrealized Gain/Loss"] / row["Quantity"])
        max_quantity = int(remaining_offset / unrealized_loss_per_unit)

        if abs(row["Unrealized Gain/Loss"]) <= remaining_offset:
            tlh_candidates.at[i, "Recommended"] = True
            tlh_candidates.at[i, "Partial Quantity"] = row["Quantity"]
            remaining_offset -= abs(row["Unrealized Gain/Loss"])
        elif max_quantity > 0:
            tlh_candidates.at[i, "Recommended"] = True
            tlh_candidates.at[i, "Partial Quantity"] = max_quantity
            remaining_offset = 0

    return tlh_candidates


def calculate_realized_gains(transactions, customer_id, tax_year):
    """
    Calculate realized gains for sell transactions in a given tax year using FIFO logic.

    Args:
        transactions (DataFrame): Full transaction data.
        customer_id (str): Customer ID for filtering.
        tax_year (int): Tax year for capital gains calculation.

    Returns:
        float: Total realized gains for the tax year.
    """
    # Filter transactions for the customer and the given tax year
    transactions = transactions[transactions["Customer ID"] == customer_id]
    transactions["Transaction Date"] = pd.to_datetime(transactions["Transaction Date"])
    sell_transactions = transactions[
        (transactions["Transaction Type"] == "Sell") &
        (transactions["Transaction Date"].dt.year == tax_year)
    ]
    
    # Group buy transactions for FIFO matching
    buy_transactions = transactions[
        (transactions["Transaction Type"] == "Buy")
    ].sort_values(by="Transaction Date")

    realized_gains = 0

    # FIFO logic for calculating gains
    for _, sell in sell_transactions.iterrows():
        ticker = sell["Ticker"]
        quantity_to_sell = sell["Quantity"]
        sell_price = sell["Price"]

        for _, buy in buy_transactions[buy_transactions["Ticker"] == ticker].iterrows():
            if quantity_to_sell <= 0:
                break

            buy_quantity = buy["Quantity"]
            buy_price = buy["Price"]
            matched_quantity = min(quantity_to_sell, buy_quantity)
            realized_gains += (sell_price - buy_price) * matched_quantity
            quantity_to_sell -= matched_quantity

            # Adjust remaining buy quantity
            buy_transactions.loc[buy.name, "Quantity"] -= matched_quantity

    return realized_gains


def calculate_sales_and_gains_summary(transactions, customer_id, tax_year):
    """
    Calculate total sales, realized gains, percent return, and top 3 stocks by return for a given tax year using FIFO logic.

    Args:
        transactions (DataFrame): Full transaction data.
        customer_id (str): Customer ID for filtering.
        tax_year (int): Tax year for calculations.

    Returns:
        str: Formatted summary string with sales, realized gains, percent return, and top 3 stocks.
    """
    # Filter transactions for the customer and the given tax year
    transactions = transactions[transactions["Customer ID"] == customer_id]
    transactions["Transaction Date"] = pd.to_datetime(transactions["Transaction Date"])
    sell_transactions = transactions[
        (transactions["Transaction Type"] == "Sell") &
        (transactions["Transaction Date"].dt.year == tax_year)
    ]

    # Group buy transactions for FIFO matching
    buy_transactions = transactions[
        (transactions["Transaction Type"] == "Buy")
    ].sort_values(by="Transaction Date")

    total_sales = 0
    realized_gains = 0
    stock_returns = []

    # FIFO logic for calculating gains
    for _, sell in sell_transactions.iterrows():
        ticker = sell["Ticker"]
        quantity_to_sell = sell["Quantity"]
        sell_price = sell["Price"]

        total_sale_for_ticker = 0
        total_cost_for_ticker = 0

        for _, buy in buy_transactions[buy_transactions["Ticker"] == ticker].iterrows():
            if quantity_to_sell <= 0:
                break

            buy_quantity = buy["Quantity"]
            buy_price = buy["Price"]
            matched_quantity = min(quantity_to_sell, buy_quantity)

            # Calculate realized gains
            realized_gains += (sell_price - buy_price) * matched_quantity

            # Track sales and costs for this ticker
            total_sale_for_ticker += sell_price * matched_quantity
            total_cost_for_ticker += buy_price * matched_quantity

            quantity_to_sell -= matched_quantity

            # Adjust remaining buy quantity
            buy_transactions.loc[buy.name, "Quantity"] -= matched_quantity

        # Record return for the ticker if sales occurred
        if total_cost_for_ticker > 0:
            percent_return = ((total_sale_for_ticker - total_cost_for_ticker) / total_cost_for_ticker) * 100
            stock_returns.append((ticker, percent_return, total_sale_for_ticker))

        # Add to total sales
        total_sales += total_sale_for_ticker

    # Get top 3 stocks by return
    top_3_stocks = sorted(stock_returns, key=lambda x: x[1], reverse=True)[:3]

    # Calculate overall percent return
    overall_percent_return = ((realized_gains / total_sales) * 100) if total_sales > 0 else 0

    # Build the formatted summary string
    summary = f"Total Sales Made: ${total_sales:,.2f}\n"
    summary += f"Realized Gains: ${realized_gains:,.2f}\n"
    summary += f"Overall Percent Return: {overall_percent_return:.2f}%\n"
    summary += "Top 3 Stocks by Return:\n"
    for ticker, percent_return, total_sales in top_3_stocks:
        summary += f"  {ticker}: {percent_return:.2f}% return, Total Sales: ${total_sales:,.2f}\n"

    return summary



### Wash Sale Detection

In [6]:
def detect_wash_sales(transactions, tlh_candidates):
    """
    Detect wash sale violations for TLH candidates and provide detailed warnings.

    Args:
        transactions (DataFrame): Transaction data containing all trades.
        tlh_candidates (DataFrame): TLH candidates being evaluated.

    Returns:
        Tuple: (List of detailed warnings, List of tickers with wash-sale violations)
    """
    warnings = []
    wash_sale_tickers = []  # To store tickers with wash sale warnings

    for _, row in tlh_candidates.iterrows():
        ticker = row["Ticker"]
        sale_date = datetime.now()  # Assume current date for sale
        # Filter transactions for recent buys within the last 30 days
        recent_buys = transactions[
            (transactions["Ticker"] == ticker) &
            (transactions["Transaction Type"] == "Buy") &
            (transactions["Transaction Date"] >= sale_date - timedelta(days=30))
        ]
        if not recent_buys.empty:
            # Find the most recent purchase date
            most_recent_buy_date = recent_buys["Transaction Date"].max()
            safe_sell_date = most_recent_buy_date + timedelta(days=31)
            warnings.append(
                f"Wash sale warning: Recent buy detected for {ticker} on {most_recent_buy_date.date()}. "
                f"You can safely sell after {safe_sell_date.date()} to avoid violations."
            )
            wash_sale_tickers.append(ticker)  # Add ticker to the list

    return warnings, wash_sale_tickers

### Reinvestment Recommendation System

In [7]:
def identify_stocks_to_sell(portfolio, customer_preferences):
    """
    Identify stocks to sell based on risk tolerance, sectors, and investment goals.
    """
    # Extract preferences
    risk_tolerance = customer_preferences["Risk Tolerance"]
    preferred_sectors = customer_preferences["Preferred Sectors"]

    # Convert sectors to a list if it's a string
    if isinstance(preferred_sectors, str):
        preferred_sectors = [sector.strip() for sector in preferred_sectors.split(",")]

    # Filter based on risk tolerance
    if risk_tolerance == "Low":
        portfolio = portfolio[portfolio["Beta"] < 1]
    elif risk_tolerance == "Moderate":
        portfolio = portfolio[(portfolio["Beta"] >= 1) & (portfolio["Beta"] <= 1.5)]
    elif risk_tolerance == "High":
        portfolio = portfolio[portfolio["Beta"] > 1.5]

    # Filter non-preferred sectors
    portfolio = portfolio[~portfolio["Sector"].isin(preferred_sectors)]

    return portfolio

def recommend_reinvestment_stocks_with_details(available_balance, stock_data, wash_sale_tickers, customer_preferences, max_stock_allocation_pct=0.15, max_sector_allocation_pct=0.30):
    """
    Recommend stocks for reinvestment while providing detailed information like purchase quantity, current price, YTD returns, and other metrics.
    """
    latest_prices = get_latest_prices(historical_prices)
    latest_prices = pd.DataFrame.from_dict(latest_prices, orient="index", columns=["Current Price"]).reset_index().rename(columns={"index": "Ticker"})
    stock_data = stock_data.merge(latest_prices, on="Ticker", how="left")

    # Exclude wash-sale-triggering stocks
    stock_data = stock_data[~stock_data["Ticker"].isin(wash_sale_tickers)]
    
    stock_data = stock_data[stock_data["YTD Return (%)"] > 0]

    # Convert preferred sectors to a list if it's a string
    preferred_sectors = customer_preferences["Preferred Sectors"]
    if isinstance(preferred_sectors, str):
        preferred_sectors = [sector.strip() for sector in preferred_sectors.split(",")]

    # Filter by sector preference
    stock_data = stock_data[stock_data["Sector"].isin(preferred_sectors)]

    # Filter by risk tolerance
    risk_tolerance = customer_preferences["Risk Tolerance"]
    if risk_tolerance == "Low":
        stock_data = stock_data[stock_data["Beta"] < 1]
    elif risk_tolerance == "Moderate":
        stock_data = stock_data[(stock_data["Beta"] >= 1) & (stock_data["Beta"] <= 1.5)]
    elif risk_tolerance == "High":
        stock_data = stock_data[stock_data["Beta"] > 1.5]

    # Rank stocks by customer goals
    if customer_preferences["Investment Goals"] == "Dividend Income":
        stock_data = stock_data.sort_values(by="Dividend Yield (%)", ascending=False)
    elif customer_preferences["Investment Goals"] == "Growth":
        stock_data = stock_data.sort_values(by="Growth Rate", ascending=False)

    # Allocation logic
    allocation = []
    sector_allocations = {}

    for _, row in stock_data.iterrows():
        # Calculate max allocations
        max_stock_allocation = available_balance * max_stock_allocation_pct
        max_sector_allocation = available_balance * max_sector_allocation_pct

        # Get current sector allocation
        current_sector_allocation = sector_allocations.get(row["Sector"], 0)
        remaining_sector_allocation = max_sector_allocation - current_sector_allocation

        # Allocate funds while respecting constraints
        allocation_amount = min(max_stock_allocation, remaining_sector_allocation, available_balance)
        if allocation_amount > 0:
            # Calculate purchase quantity
            purchase_quantity = allocation_amount // row["Current Price"]
            allocation_amount = purchase_quantity * row["Current Price"]  # Recalculate based on whole shares

            allocation.append({
                "Ticker": row["Ticker"],
                "Allocation": allocation_amount,
                "Purchase Quantity": purchase_quantity,
                "Current Price": row["Current Price"],
                "YTD Return": row["YTD Return (%)"],
                "P/E Ratio": row.get("P/E Ratio", "N/A"),
                "Market Cap (B)": row.get("Market Cap (B)", "N/A"),
                "Beta": row.get("Beta", "N/A"),
                "52-Week High": row.get("52-Week High", "N/A"),
                "52-Week Low": row.get("52-Week Low", "N/A"),
                "Growth Rate": row.get("Growth Rate","N/A"),
                "Projected Growth": allocation_amount * (1 + row["Growth Rate"])
            })

            # Update balances
            available_balance -= allocation_amount
            sector_allocations[row["Sector"]] = current_sector_allocation + allocation_amount

        # Stop if all funds are allocated
        if available_balance <= 0:
            break

    # Redistribute residual balance proportionally if any remains
    if available_balance > 0:
        total_allocation = sum(a["Allocation"] for a in allocation)
        for entry in allocation:
            proportional_addition = (entry["Allocation"] / total_allocation) * available_balance
            entry["Allocation"] += proportional_addition
            entry["Purchase Quantity"] += int(proportional_addition // entry["Current Price"])
            entry["Projected Growth"] = entry["Allocation"] * (1 + stock_data.loc[stock_data["Ticker"] == entry["Ticker"], "Growth Rate"].values[0])
    
    # Filter out zero-allocation stocks
    allocation_df = pd.DataFrame(allocation)
    allocation_df = allocation_df[allocation_df["Allocation"] > 0]
    allocation_df = allocation_df.drop(columns=["Allocation"], errors="ignore")

    return allocation_df



### Execute code and Convert all data to JSON for Watsonx Discovery

In [8]:
import pandas as pd
import json
import re

# Assume all necessary modules and functions are already imported
# and original_transactions, customer_profiles, historical_prices,
# tax_rates, stock_expanded_data, etc., are already defined.

all_customers_data = []

# Function to reset data before processing each customer
def reset_data():
    global transactions
    transactions = original_transactions.copy()
    # Repeat for other DataFrames if necessary

# Function to process a single customer
def process_customer(customer_id):
    # Reset data before processing each customer
    reset_data()
    
    tax_year = 2024  # Use the current or target tax year

    # Extract the customer's profile
    customer_profile = customer_profiles[customer_profiles["Customer ID"] == customer_id].iloc[0]
    
    # Extract user preferences
    risk_tolerance = customer_profile["Risk Tolerance"]
    preferred_sectors = customer_profile["Preferred Sectors"].split(", ")
    investment_goals = customer_profile["Investment Goals"]
    current_prices = get_latest_prices(historical_prices)
    income = customer_profile["Annual Income"]
    
    # Calculate the customer's portfolio
    portfolio = compute_fifo_portfolio_with_metrics(transactions, current_prices, customer_id)
    
    # Identify TLH candidates
    tlh_candidates = identify_tlh_candidates(portfolio)
    
    # Calculate tax savings for TLH candidates
    tlh_candidates_with_savings = calculate_tax_savings(tlh_candidates, income, tax_rates)
    
    # Calculate realized gains dynamically from transactions
    realized_gains = calculate_realized_gains(transactions, customer_id, tax_year)
    
    # Generate final TLH recommendations
    final_tlh_recommendations = finalize_tlh_recommendation(
        tlh_candidates_with_savings,
        realized_gains,
        max_loss_offset=3000
    )
    
    # Detect potential wash sale violations
    wash_sale_warnings, wash_sale_tickers = detect_wash_sales(transactions, tlh_candidates_with_savings)
    
    # Calculate the sale proceeds for reinvestment
    final_tlh_recommendations["Sale Proceeds"] = (
        final_tlh_recommendations["Partial Quantity"] * final_tlh_recommendations["Current Price"]
    )
    available_balance = final_tlh_recommendations["Sale Proceeds"].sum()
    
    # Identify stocks to sell (if applicable)
    # stocks_to_sell = identify_stocks_to_sell(stock_expanded_data, customer_profile)
    
    # Filter stocks based on customer preferences
    recommended_stocks = recommend_reinvestment_stocks_with_details(
        available_balance, stock_expanded_data, wash_sale_tickers, customer_profile
    )
    
    # --- Portfolio Summary ---
    portfolio_updated = portfolio.drop(columns=["Holding Period (Days)"])
    going_well = portfolio_updated[portfolio_updated["Unrealized Gain/Loss"] > 0]
    
    going_bad = portfolio_updated[portfolio_updated["Unrealized Gain/Loss"] < 0].copy()
    going_bad['Unrealized Gain/Loss'] = going_bad['Unrealized Gain/Loss'].abs()
    going_bad.rename(columns={'Unrealized Gain/Loss':'Unrealized Loss'}, inplace=True)
    going_well.rename(columns={'Unrealized Gain/Loss':'Unrealized Gain'}, inplace=True)
    
    # Calculate Total Investments
    portfolio["Initial Investment"] = portfolio["Purchase Price"] * portfolio["Quantity"]
    portfolio["Current Value"] = portfolio["Current Price"] * portfolio["Quantity"]
    total_initial_investment = portfolio["Initial Investment"].sum()
    total_current_value = portfolio["Current Value"].sum()
    net_unrealized_gain_loss_percent = (
        (portfolio["Unrealized Gain/Loss"].sum() / portfolio["Initial Investment"].sum()) * 100
    ).round(2)
    
    # Sales made in the tax year
    customer_transactions = transactions[transactions["Customer ID"] == customer_id].copy()
    customer_transactions["Transaction Date"] = pd.to_datetime(customer_transactions["Transaction Date"])
    sell_transactions = customer_transactions[
        (customer_transactions["Transaction Type"] == "Sell") &
        (customer_transactions["Transaction Date"].dt.year == tax_year)
    ]
    
    # Calculate sales and gains summary
    summary = calculate_sales_and_gains_summary(transactions, customer_id, tax_year)
    total_sales = re.search(r"Total Sales Made: \$(\d[\d,\.]*)", summary).group(1)
    realized_gains_str = re.search(r"Realized Gains: \$(\d[\d,\.]*)", summary).group(1)
    overall_percent_return = re.search(r"Overall Percent Return: ([\d\.]*)%", summary).group(1)
    top_stocks = re.findall(r"(\w+): ([\d\.]+)% return, Total Sales: \$(\d[\d,\.]*)", summary)
    realized_gains_float = float(realized_gains_str.replace(",", "")) if isinstance(realized_gains_str, str) else float(realized_gains_str)
    
    # --- Tax Loss Harvesting Analysis ---
    # Generate TLH Table with Additional Columns
    tlh_table = final_tlh_recommendations[[
        "Ticker", "Unrealized Gain/Loss", "Partial Quantity", "Current Price", "Recommended", "Sale Proceeds"
    ]].copy()
    
    # Calculate Absolute Losses
    tlh_table["Unrealized Gain/Loss"] = tlh_table["Unrealized Gain/Loss"].abs()
    tlh_table.rename(columns={
        "Unrealized Gain/Loss": "Losses",
        "Partial Quantity": "Quantity to Sell"
    }, inplace=True)
    
    # Calculate Total Tax Savings
    total_tax_savings = final_tlh_recommendations[final_tlh_recommendations["Recommended"]]["Tax Savings"].sum()
    
    # Calculate Total Sale Proceeds from TLH Sales
    total_sale_proceeds = final_tlh_recommendations[final_tlh_recommendations["Recommended"]]["Sale Proceeds"].sum()
    
    # Generate Wash Sale Output
    wash_sale_output = "\n".join(wash_sale_warnings) if wash_sale_warnings else "No wash sale warnings."
    
    # Calculate Total Offset Limit
    total_offset_limit = realized_gains_float + 3000 if realized_gains_float > 0 else 3000
    
    # --- Reinvestment Recommendations ---
    # Reinvest amount
    total_investment = (recommended_stocks["Purchase Quantity"] * recommended_stocks["Current Price"]).sum()
    
    # Add Allocation (%) column
    recommended_stocks["Allocation (%)"] = (
        (recommended_stocks["Purchase Quantity"] * recommended_stocks["Current Price"]) / total_investment * 100
    )
    
    # Calculate Weighted YTD Return
    weighted_ytd_return = (
        (recommended_stocks["Purchase Quantity"] * recommended_stocks["YTD Return"]).sum() /
        recommended_stocks["Purchase Quantity"].sum()
    )
    
    # Calculate Portfolio Beta
    portfolio_beta = (
        (recommended_stocks["Purchase Quantity"] * recommended_stocks["Beta"]).sum() /
        recommended_stocks["Purchase Quantity"].sum()
    )
    
    # Calculate Average P/E Ratio
    average_pe_ratio = recommended_stocks["P/E Ratio"].mean()
    
    # Calculate Average Growth Rate
    average_growth_rate = recommended_stocks["Growth Rate"].mean()
    
    # Create the Recommendations Table
    recommendations_table = recommended_stocks[
        ["Ticker", "Purchase Quantity", "Current Price", "YTD Return", "Beta", "P/E Ratio", "Growth Rate", "Allocation (%)"]
    ].copy()
    
    # Round numbers where needed
    recommendations_table["Allocation (%)"] = recommendations_table["Allocation (%)"].round(2)
    weighted_ytd_return = round(weighted_ytd_return, 2)
    portfolio_beta = round(portfolio_beta, 2)
    average_pe_ratio = round(average_pe_ratio, 2)
    average_growth_rate = round(average_growth_rate, 2)
    recommendations_table["YTD Return"] = recommendations_table["YTD Return"].round(2)
    recommendations_table["Growth Rate"] = recommendations_table["Growth Rate"].round(2)
    recommendations_table["Current Price"] = recommendations_table["Current Price"].round(2)
    recommendations_table["P/E Ratio"] = recommendations_table["P/E Ratio"].round(2)
    recommendations_table["Beta"] = recommendations_table["Beta"].round(2)
    
    # --- Executive Summary ---
    net_unrealized_gain_loss = portfolio["Unrealized Gain/Loss"].sum().round(2)
    recommendations_stocks = recommended_stocks["Ticker"]
    expected_average_growth_rate = average_growth_rate
    
    # --- Convert all data to JSON ---
    output_data = {
        "Customer ID": customer_id,
        "Executive Summary": {
            "Customer Profile": {
                "Risk Tolerance": customer_profile["Risk Tolerance"],
                "Preferred Sectors": customer_profile["Preferred Sectors"],
                "Investment Goals": customer_profile["Investment Goals"],
            },
            "Portfolio Summary": {
                "Total Current Value": round(total_current_value, 2),
                "Net Unrealized Gain/Loss": round(net_unrealized_gain_loss, 2),
            },
            "Tax-Loss Harvesting": {
                "Total Tax Savings": round(total_tax_savings, 2),
                "Reinvestment Amount": round(total_sale_proceeds, 2),
            },
            "Reinvestment Recommendations": {
                "Recommended Stocks": recommendations_stocks.tolist(),
                "Expected Average Growth Rate %": round(expected_average_growth_rate, 2),
            },
        },
        "Portfolio Summary": {
            "Total Initial Investment": round(total_initial_investment, 2),
            "Total Current Value": round(total_current_value, 2),
            "Net Unrealized Gain/Loss Percent": round(net_unrealized_gain_loss_percent, 2),
            "Going Bad Investments": [
                {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in record.items()}
            for record in going_bad.to_dict(orient="records")
            ],
            "Going Well Investments": [
                {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in record.items()}
            for record in going_well.to_dict(orient="records")
            ],
            "Sales Analysis": {
                "Total Sales Made": round(float(total_sales.replace(",", "")), 2),
                "Realized Gains": round(realized_gains_float, 2),
                "Overall Percent Return": round(float(overall_percent_return), 2),
                "Top Stocks by Return": [
                    {
                        "Ticker": stock[0],
                        "Return Percent": round(float(stock[1]), 2),
                        "Total Sales": round(float(stock[2].replace(",", "")), 2),
                    }
                    for stock in top_stocks
                ],
            },
        },
        "Tax Loss Harvesting Analysis": {
            "Table": [
                {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in record.items()}
            for record in tlh_table.to_dict(orient="records")
            ],
            "Total Tax Savings": round(total_tax_savings, 2),
            "Total Sale Proceeds from TLH Sales": round(total_sale_proceeds, 2),
            "Wash Sale Warnings": wash_sale_warnings,
        },
        "Reinvestment Recommendations": {
            "Customer Profile": {
                "Risk Tolerance": customer_profile["Risk Tolerance"],
                "Preferred Sectors": customer_profile["Preferred Sectors"],
                "Investment Goals": customer_profile["Investment Goals"],
            },
            "Table": [
                {k: round(v, 2) if isinstance(v, (int, float)) else v for k, v in record.items()}
            for record in recommendations_table.to_dict(orient="records")
            ],
            "New Portfolio Summary": {
                "Total Investment": round(total_investment, 2),
                "Weighted YTD Return": weighted_ytd_return,
                "Portfolio Beta": portfolio_beta,
                "Average P/E Ratio": average_pe_ratio,
                "Expected Average Growth Rate": average_growth_rate,
            },
        },
    }
    
    # Instead of writing to a file, append the output_data to the list
    all_customers_data.append(output_data)
    
    print(f"Processed customer {customer_id}")

# Get a list of all customer IDs
customer_ids = customer_profiles["Customer ID"].unique()

# Loop over all customer IDs and process each one
for customer_id in customer_ids:
    process_customer(customer_id)

# After processing all customers, write the accumulated data to a single JSON file
with open("all_customers_report.json", "w") as file:
    json.dump(all_customers_data, file, indent=4)

print("All customer data has been written to 'all_customers_report.json'")

Processed customer C001
Processed customer C002
All customer data has been written to 'all_customers_report.json'


/tmp/wsuser/ipykernel_438/2890460161.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  transactions["Transaction Date"] = pd.to_datetime(transactions["Transaction Date"])
/tmp/wsuser/ipykernel_438/3882620736.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  going_well.rename(columns={'Unrealized Gain/Loss':'Unrealized Gain'}, inplace=True)
/tmp/wsuser/ipykernel_438/2890460161.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

In [9]:
import json
import ibm_boto3
from ibm_botocore.client import Config

# IBM COS Credentials and Configurations
COS_API_KEY = "RuSq2hb5DI8kOl1ZznOxQHx3igYJObJF-yTHDDkNi55E"  # Replace with your IBM COS API Key
COS_ENDPOINT = "https://s3.direct.us-south.cloud-object-storage.appdomain.cloud"  # Replace with your endpoint
COS_BUCKET_NAME = "007tlhdata"  # Replace with your bucket name
COS_OBJECT_NAME = "json_customer_data/all_customers_data.json"  # Name of the file to be saved in COS

# Initialize COS client
cos_client = ibm_boto3.client(
    service_name="s3",
    ibm_api_key_id=COS_API_KEY,
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT,
)

# Convert JSON to a string
json_data = json.dumps(all_customers_data)

# Upload the JSON file to IBM COS
try:
    response = cos_client.put_object(
        Bucket=COS_BUCKET_NAME,
        Key=COS_OBJECT_NAME,
        Body=json_data,
        ContentType="application/json",
    )
    print(f"Upload Successful! ETag: {response['ETag']}")
except Exception as e:
    print(f"Error uploading file: {e}")


Upload Successful! ETag: "1abb7028705e029d39029b84725f09f8"
